In [ ]:
!pip install pandas
!pip install torch
!pip install transformers
!pip install  python-telegram-bot
!pip install pyTelegramBotAPI

In [ ]:

import pandas as pd
import torch
from transformers import DistilBertTokenizer,DistilBertForSequenceClassification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import telebot

# Load the dataset
dataset_path = "/content/water_potability.csv"
data = pd.read_csv(dataset_path)

# Replace missing values
data.fillna(data.mean(), inplace=True)

# Extract features and target
features = data.drop(columns=["Potability"]).columns.tolist()
target = "Potability"

# Train a RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(data[features], data[target])

#Load Distillbert tokenizer and model
tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_distillbert=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')


# Telegram bot setup
bot_token = '6179366681:AAGPmu2sL7hMYPSr5TvX0bGBtsLnIvtdQFE'  # Replace with your bot token
bot = telebot.TeleBot(bot_token)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome! Please enter the values for the input features:ph, Hardness, Solids, Chloramines,Sulfate, Conductivity, Organic_carbon, Trihalomethanes, Turbidity}")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    input_values = message.text.strip().split(',')

    if len(input_values) != len(features):
        bot.reply_to(message, f"Invalid input. Please enter values for all features.")
    else:
        input_features = [float(value) for value in input_values]

        potability_prediction = model_rf.predict([input_features])[0]
        potability_label = "Potable" if potability_prediction == 1 else "Not Potable"

        response = f"The water is {potability_label}"
        bot.reply_to(message, response)

bot.polling()

ERROR: Operation cancelled by user


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
